In [1]:
import pandas as pd
import os
import glob
import math
import datetime as datetime


from koku_to_cs import *

In [2]:
def appendDataframesWithGoodFormat(df1,df2):
        
    report_df = df1.append(df2)
    
    report_df["Amount"] = ''
    report_df["Currency"] = ''
    report_df["Evidence3"] = report_df["Evidence3"].astype(float)
    report_df["Evidence4"] = report_df["Evidence4"].astype(float)
    report_df["Unweighted_Volume"] = report_df["Unweighted_Volume"].astype(float)
    report_df["Evidence1"] = report_df["Evidence1"].apply(lambda x: str(x).lower())

    report_df_without_volume = report_df[report_df['Evidence2_DomainID'] != 'Application Namespace_Storage']
    report_df_without_volume = report_df_without_volume.groupby(['Consumption_Type','Sender_DomainID','Sender_Region','Sender_ObjectID','Sender_OptionID','Receiver_Office','Evidence1_DomainID','Evidence1','Evidence2_DomainID','Evidence2','Evidence3_DomainID','Evidence4_DomainID','Amount','Currency','Month','Year','Scenario','Booking_Costtype'], as_index=False).sum()
    report_df_without_volume = report_df_without_volume[['Consumption_Type','Sender_DomainID','Sender_Region','Sender_ObjectID','Sender_OptionID','Receiver_Office','Evidence1_DomainID','Evidence1','Evidence2_DomainID','Evidence2','Evidence3_DomainID','Evidence3','Evidence4_DomainID','Evidence4','Unweighted_Volume','Amount','Currency','Month','Year','Scenario','Booking_Costtype']]


    report_df_only_volume = report_df[report_df['Evidence2_DomainID'] == 'Application Namespace_Storage']
    report_df_only_volume = report_df_only_volume.groupby(['Consumption_Type','Sender_DomainID','Sender_Region','Sender_ObjectID','Sender_OptionID','Receiver_Office','Evidence1_DomainID','Evidence1','Evidence2_DomainID','Evidence2','Evidence3_DomainID','Evidence4_DomainID','Amount','Currency','Month','Year','Scenario','Booking_Costtype'], as_index=False).mean()
    report_df_only_volume = report_df_only_volume[['Consumption_Type','Sender_DomainID','Sender_Region','Sender_ObjectID','Sender_OptionID','Receiver_Office','Evidence1_DomainID','Evidence1','Evidence2_DomainID','Evidence2','Evidence3_DomainID','Evidence3','Evidence4_DomainID','Evidence4','Unweighted_Volume','Amount','Currency','Month','Year','Scenario','Booking_Costtype']]


    appended_report = report_df_without_volume.append(report_df_only_volume)

    return appended_report

In [ ]:
# Loading files generated by Koku after unzipping them

path = '../reports/upload/'
files = glob.glob(path + '/*.csv')

'''
Not First day of the month.
1fd80ead-0244-4464-8815-f561d1f7bf54_openshift_usage_report.0.csv 0 -> namespace information
1fd80ead-0244-4464-8815-f561d1f7bf54_openshift_usage_report.1.csv 1 -> node information
1fd80ead-0244-4464-8815-f561d1f7bf54_openshift_usage_report.2.csv 2 -> pod information
1fd80ead-0244-4464-8815-f561d1f7bf54_openshift_usage_report.3.csv 3 -> volume information

First day of the month
56dc60a4-e6d8-4350-8126-5e6b3f10b7ad_openshift_usage_report.0.csv # Previous Month
56dc60a4-e6d8-4350-8126-5e6b3f10b7ad_openshift_usage_report.1.csv # Current Month
56dc60a4-e6d8-4350-8126-5e6b3f10b7ad_openshift_usage_report.2.csv # Previous Month
56dc60a4-e6d8-4350-8126-5e6b3f10b7ad_openshift_usage_report.3.csv # Current Month
56dc60a4-e6d8-4350-8126-5e6b3f10b7ad_openshift_usage_report.4.csv # Previous Month
56dc60a4-e6d8-4350-8126-5e6b3f10b7ad_openshift_usage_report.5.csv # Current Month
56dc60a4-e6d8-4350-8126-5e6b3f10b7ad_openshift_usage_report.6.csv # Previous Month
56dc60a4-e6d8-4350-8126-5e6b3f10b7ad_openshift_usage_report.7.csv # Current Month
'''

namespace_path = glob.glob(path + '/*0.csv')[0]
node_path = glob.glob(path + '/*2.csv')[0]
pod_path = glob.glob(path + '/*4.csv')[0]
volume_path = glob.glob(path + '/*6.csv')[0]


namespace_data_raw = pd.read_csv(namespace_path)
node_data_raw = pd.read_csv(node_path)
node_data_raw_bis = pd.read_csv(node_path)
pod_data_raw = pd.read_csv(pod_path)
volume_data_raw = pd.read_csv(volume_path)

In [2]:
# Loading files generated by Koku after unzipping them
isfile = os.path.isfile
path = '../reports_new/upload/'
files = glob.glob(path + '/*.csv')

# Checking how many file there was in the tar file from the koku report.
# Normally there should be 4 csv files each corresponding to namespace data, node data, pod data, volume data
if len(files) == 4:
    namespace_path = glob.glob(path + '/*0.csv')[0]
    node_path = glob.glob(path + '/*1.csv')[0]
    pod_path = glob.glob(path + '/*2.csv')[0]
    volume_path = glob.glob(path + '/*3.csv')[0]


    namespace_data_raw = pd.read_csv(namespace_path)
    node_data_raw = pd.read_csv(node_path)
    node_data_raw_bis = pd.read_csv(node_path)
    pod_data_raw = pd.read_csv(pod_path)
    volume_data_raw = pd.read_csv(volume_path)
    
    metering_report_df = koku_to_cs(namespace_data_raw, node_data_raw, pod_data_raw, volume_data_raw)
    
    month = getMonth(namespace_data_raw)
    report_path = '../CS_billing_reports/openshift_usage_month_{}_billing_report.csv'.format(month)
    
    if isfile(report_path):
    
        report_df = pd.read_csv(report_path)

        appended_report = appendDataframesWithGoodFormat(report_df,metering_report_df)    
        appended_report.to_csv(report_path, index=False)
    
    else:
        metering_report_df.to_csv(report_path, index=False)
    
    
    
# if there are 8 csv files, it should mean that the koku metering operator report have metrics for the last day of
# a given month and metrics for the first day of a new month. 4 csv files for end of the month metrics, 4 csv files
# for new month metrics.
elif len(files) == 8:
    
    # check that the file are in the correct order
    previous_month_path = glob.glob(path + '/*0.csv')[0]
    current_month_path = glob.glob(path + '/*1.csv')[0]
    
    namespace_data_previous_month = pd.read_csv(previous_month_path)
    namespace_data_current_month = pd.read_csv(current_month_path)
    
    if set(namespace_data_previous_month.columns) == set(namespace_data_current_month.columns):
        
        # Previous Month
        namespace_previous_month_path = glob.glob(path + '/*0.csv')[0]
        node_previous_month_path = glob.glob(path + '/*2.csv')[0]
        pod_previous_month_path = glob.glob(path + '/*4.csv')[0]
        volume_previous_month_path = glob.glob(path + '/*6.csv')[0]
        
        namespace_previous_month_data_raw = pd.read_csv(namespace_previous_month_path)
        node_previous_month_data_raw = pd.read_csv(node_previous_month_path)
        pod_previous_month_data_raw = pd.read_csv(pod_previous_month_path)
        volume_previous_month_data_raw = pd.read_csv(volume_previous_month_path)
        
        metering_previous_month_report_df = koku_to_cs(namespace_previous_month_data_raw, node_previous_month_data_raw, pod_previous_month_data_raw, volume_previous_month_data_raw)

        month = getMonth(namespace_previous_month_data_raw)
        report_path = '../CS_billing_reports/openshift_usage_month_{}_billing_report.csv'.format(month)

        if isfile(report_path):
            report_df = pd.read_csv(report_path)

            appended_report = appendDataframesWithGoodFormat(report_df,metering_previous_month_report_df)    
            appended_report.to_csv(report_path, index=False)
        
        else:
            metering_previous_month_report_df.to_csv(report_path, index=False)
        
        # New Month
        namespace_current_month_path = glob.glob(path + '/*1.csv')[0]
        node_current_month_path = glob.glob(path + '/*3.csv')[0]
        pod_current_month_path = glob.glob(path + '/*5.csv')[0]
        volume_current_month_path = glob.glob(path + '/*7.csv')[0]
        
        namespace_current_month_data_raw = pd.read_csv(namespace_current_month_path)
        node_current_month_data_raw = pd.read_csv(node_current_month_path)
        pod_current_month_data_raw = pd.read_csv(pod_current_month_path)
        volume_current_month_data_raw = pd.read_csv(volume_current_month_path)
    
        metering_current_month_report_df = koku_to_cs(namespace_current_month_data_raw, node_current_month_data_raw, pod_current_month_data_raw, volume_current_month_data_raw)

        month = getMonth(namespace_current_month_data_raw)
        report_path = '../CS_billing_reports/openshift_usage_month_{}_billing_report.csv'.format(month)

        if isfile(report_path):
    
            report_df = pd.read_csv(report_path)

            appended_report = appendDataframesWithGoodFormat(report_df,metering_current_month_report_df)    
            appended_report.to_csv(report_path, index=False)
            
        else:
            metering_current_month_report_df.to_csv(report_path, index=False)
    else:
        raise Exception('Koku reports file naming is incorrect \n Note: In a new month, if there is metering data from the previous month and the new month, the following file naming is expected: \n <uid>_openshift_usage_report.0.csv  Namespace data Previous Month \n <uid>_openshift_usage_report.1.csv  Namespace data Current Month \n <uid>_openshift_usage_report.2.csv  Node data Previous Month \n <uid>_openshift_usage_report.3.csv  Node data Current Month \n <uid>_openshift_usage_report.4.csv  Pod data Previous Month \n <uid>_openshift_usage_report.5.csv Pod data Current Month \n <uid>_openshift_usage_report.6.csv  Volume data Previous Month \n <uid>_openshift_usage_report.7.csv  Volume data Current Month')
        
else:
    raise Exception('There are not enough or too many reports for the script to run. \n Note: In a koku reports tar file, there should be 4 reports and on a new month if there is report month overlapping, there should be 8 reports')

/home/edubois/RedHat/Engagements/CreditSuisse/2021/koku-metrics-operator/jupyterNotebooks/koku_to_cs.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  namespace_data["namespace_labels"] = namespace_data["namespace_labels"].astype(str)
/home/edubois/RedHat/Engagements/CreditSuisse/2021/koku-metrics-operator/jupyterNotebooks/koku_to_cs.py:370: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  volume_request_storage_gigabytes = volume_data.loc[((volume_data['namespace'] == namespace) & (volume_data['volume_request_storage_gigabytes'] > 0))].mean().volume_request_storage_gigabytes
